In [140]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import string

from util import randomChoice, lineToTensor, tensorToLine

device = torch.device("cpu")

datensatz = [
    ["MAN","KING"],
    ["WOMAN","QUEEN"]
]

In [141]:
class word2vec(nn.Module):
    def __init__(self, input_size, output_size):
        super(word2vec, self).__init__()
        
        self.i2o_L1 = nn.Linear(input_size, math.ceil(input_size / 2))
        self.i2o_L2 = nn.Linear(math.ceil(input_size/ 2), math.ceil(input_size / 4))
        self.i2o_L3 = nn.Linear(math.ceil(input_size / 4), math.ceil(input_size / 2))
        self.i2o_L4 = nn.Linear(math.ceil(input_size / 2), output_size)
        
    def forward(self, input):
        output = self.i2o_L1(input)
        output = self.i2o_L2(output)
        output = self.i2o_L3(output)
        output = self.i2o_L4(output)
        return output
    
    def encode(self, input):
        output = self.i2o_L1(input)
        output = self.i2o_L2(output)
        return output
    
    def decode(self, input):
        output = self.i2o_L3(input)
        output = self.i2o_L4(output)
        return output

In [142]:
all_letters = string.ascii_letters + " .,;'" + "äÄüÜöÖ"
n_letters = len(all_letters)
contextLength = 2

model = word2vec(context_length(n_letters * 10), n_letters * 10).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [143]:
learning_rate = 0.005

In [144]:
def trainStep(line_tensor):
    model.zero_grad()

    output = model(line_tensor.to(device))

    loss = F.mse_loss(output, line_tensor.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return output, loss.item()

In [159]:
current_loss = 0
for iter in range(0, 5000):
    for line in datensatz:
        line_tensor = lineToTensor(line, 10).flatten()
        output, loss = trainStep(line_tensor)
        current_loss += loss
    print(current_loss, end="\r")
    current_loss = 0

In [160]:
king = model.encode(lineToTensor("King",10).flatten())

In [161]:
man = model.encode(lineToTensor("Man",10).flatten())

In [162]:
woman = model.encode(lineToTensor("Woman",10).flatten())

In [163]:
queen = woman + man

In [164]:
word = model.decode(queen).detach()

In [165]:
tensorToLine(torch.abs(torch.round(word)).view(10, n_letters))

'MWaomnan'